# Popular Data Science Questions
In this project, we assume the role of Data Analysts working in a company that creates Data Science content. The content can be in form of books, online articles, videos or interactive text. We have been tasked with figuring out the best content to write about for maximum user engagement.

To address this task, we decide to collect data from **[stack exchange](https://en.wikipedia.org/wiki/Stack_Exchange)**, a network of question-and-answer websites. Stack Exchange hosts sites on a multitude of fields and subjects, including Mathematics, Physics, Statistics, and Data Science! Our rationale is simple: If we can identify Data Science questions that people commonly ask, then we can tailor our content to address those topics.

## Why Stack Exchange?
---
1. **Relevance:** Data Science is a multidisciplinary field and Stack Exchange provides a couple of websites that are relevant to our goal. Examples include:
>- [Data Science](https://datascience.stackexchange.com/)
>- [Cross Validated](https://stats.stackexchange.com/) — a statistics site
>- [Artificial Intelligence](https://ai.stackexchange.com/)
>- [Mathematics](https://math.stackexchange.com/)
>- [Stack Overflow](https://stackoverflow.com/)
  
2. **It is easy to identify good posts**: Stack Exchange uses a reputation award system for its questions and answers. Each post (question or answer) is subject to upvotes and downvotes. This makes it easy to identify post that users engage and love.<br><br>
3. **Established posting guidelines**: Stack Exchange's Data Science help center states that questions should be objective, practical about Data Science, specific and relevant to other users.

These attributes make Stack Exchange a good source of data for achieving our goal.